In [18]:
import pyspark
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
NESSIE_URI = "http://nessie:19120/api/v1"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"



conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
        .set('spark.sql.catalog.nessie.s3.endpoint', 'http://minio:9000')
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
  		#MINIO CREDENTIALS
        .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

Spark Running


In [19]:
spark.sql("CREATE TABLE IF NOT EXISTS nessie.test1(name string) USING iceberg").show()

++
||
++
++



In [20]:
spark.sql("insert into nessie.test1 values('test')")

DataFrame[]

In [36]:
spark.sql("select * from nessie.test1").show()

+----------+-------------------+
|      name|                 ts|
+----------+-------------------+
|      test|2025-01-01 00:00:00|
|iceberg v2|2025-02-02 00:00:00|
+----------+-------------------+



In [29]:
spark.sql("INSERT INTO nessie.test1 VALUES ('iceberg v2')")

DataFrame[]

In [35]:
spark.sql("SELECT * FROM nessie.test1.snapshots").show(truncate=True)


+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2025-04-15 14:57:...|4482891011596568820|               null|   append|s3a://warehouse/t...|{spark.app.id -> ...|
|2025-04-15 20:30:...|4771168873951498322|4482891011596568820|   append|s3a://warehouse/t...|{spark.app.id -> ...|
|2025-04-16 09:37:...| 359364544268001682|4482891011596568820|   append|s3a://warehouse/t...|{spark.app.id -> ...|
|2025-04-16 09:37:...|3491015018832812929| 359364544268001682|   append|s3a://warehouse/t...|{spark.app.id -> ...|
|2025-04-16 09:38:...|2090234595394550542|3491015018832812929|   append|s3a://warehouse/t...|{spark.app.id -> ...|
|2025-04-16 09:38:...|6687891876915979396|4482891011596568820|   append|s3a://wa

In [24]:
## time travel to old snapshot

df = spark.read.option("snapshot-id", 4482891011596568820) \
    .format("iceberg") \
    .load("nessie.test1")

df.show()

+----+
|name|
+----+
|test|
+----+



In [27]:
spark.sql("""
    CALL nessie.system.rollback_to_snapshot(
        'test1',
        4482891011596568820
    )
""")

25/04/16 09:38:14 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


DataFrame[previous_snapshot_id: bigint, current_snapshot_id: bigint]